# Pseudochannel Explorer

Interactive tool for creating weighted pseudochannel composites from multiplex tissue imaging data.

## Workflow
1. Configure paths below
2. Load and explore channels interactively
3. Preview segmentation on zoom regions (optional)
4. Save weights and apply to full images

## Setup

In [ ]:
import sys
from pathlib import Path

# Add src to path if running from notebooks folder
src_path = Path("../src").resolve()
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

from pseudochannel import (
    DEFAULT_EXCLUDED_CHANNELS,
    FolderChannels,
    OMETiffChannels,
    compute_pseudochannel,
    detect_input_mode,
    load_channel_folder,
    save_config,
    load_config,
    find_mcmicro_experiments,
    process_mcmicro_batch,
)
from pseudochannel.widgets import create_interactive_explorer
from pseudochannel.config import get_weights_from_config, list_configs
from pseudochannel.preview import downsample_image
from pseudochannel.batch import batch_process_directory, process_ome_tiff_batch

import numpy as np
import matplotlib.pyplot as plt
import tifffile

%matplotlib widget

In [ ]:
# ==== CONFIGURATION ====

# Option A: Folder of individual channel TIFFs
CHANNEL_FOLDER = None
MACSIMA_MODE = True  # Auto-detect DAPI, use MACSima naming

# Option B: OME-TIFF with marker file
OME_TIFF_PATH = None
MARKER_FILE = None
MARKER_COLUMN = None      # Column name/index for CSV, or None for plain text
MCMICRO_MARKERS = False   # True for MCMICRO format (filters remove=TRUE)

# If both options provided, prefer OME-TIFF?
PREFER_OME_TIFF = False

# Channel exclusions: None=defaults, []=none, ["DAPI","X"]=specific
EXCLUDE_CHANNELS = None

# Nuclear marker: None=auto-detect (MACSima/OME-TIFF), or explicit path
NUCLEAR_MARKER_PATH = None

# Output paths
CONFIG_DIR = Path("../configs")
OUTPUT_DIR = Path("../outputs")

## Load Channels

In [ ]:
INPUT_MODE = detect_input_mode(CHANNEL_FOLDER, OME_TIFF_PATH, MARKER_FILE, PREFER_OME_TIFF)
print(f"Input mode: {INPUT_MODE}")

In [ ]:
if INPUT_MODE == 'folder':
    channels = load_channel_folder(
        CHANNEL_FOLDER,
        exclude_channels=EXCLUDE_CHANNELS,
        macsima_mode=MACSIMA_MODE,
    )
    if isinstance(channels, FolderChannels) and channels.nuclear_path:
        print(f"Auto-detected nuclear marker: {channels.nuclear_path.name}")
else:
    channels = OMETiffChannels(
        OME_TIFF_PATH,
        MARKER_FILE,
        marker_column=MARKER_COLUMN,
        exclude_channels=EXCLUDE_CHANNELS,
        mcmicro_markers=MCMICRO_MARKERS,
    )

print(f"Loaded {len(channels)} channels: {', '.join(list(channels.keys())[:5])}...")

## Interactive Weight Tuning

- Drag sliders to adjust channel weights
- Draw rectangle on preview to zoom at full resolution
- Click **Segment** to preview Cellpose segmentation on zoom region
- Toggle **Show Nuclear** to overlay DAPI in blue

In [ ]:
if INPUT_MODE == 'folder':
    explorer = create_interactive_explorer(
        CHANNEL_FOLDER,
        exclude_channels=EXCLUDE_CHANNELS,
        nuclear_marker_path=NUCLEAR_MARKER_PATH,
        macsima_mode=MACSIMA_MODE,
    )
else:
    explorer = create_interactive_explorer(
        OME_TIFF_PATH,
        marker_file=MARKER_FILE,
        marker_column=MARKER_COLUMN,
        exclude_channels=EXCLUDE_CHANNELS,
        nuclear_marker_path=NUCLEAR_MARKER_PATH,
        mcmicro_markers=MCMICRO_MARKERS,
    )

In [ ]:
# Show current active weights
active = {k: v for k, v in explorer.get_weights().items() if v > 0}
print(f"Active weights ({len(active)}):")
for name, weight in sorted(active.items(), key=lambda x: -x[1]):
    print(f"  {name}: {weight:.2f}")

## Apply to Full Resolution

In [ ]:
weights = explorer.get_weights()
full_res = compute_pseudochannel(channels, weights, normalize="minmax")
print(f"Full-resolution result: {full_res.shape}")

In [ ]:
# Display downsampled preview
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(downsample_image(full_res, 1024), cmap='gray')
ax.set_title('Full Resolution (downsampled for display)')
ax.axis('off')
plt.tight_layout()

In [ ]:
# Save as 16-bit TIFF
output_path = OUTPUT_DIR / "pseudochannel_output.tif"
output_path.parent.mkdir(parents=True, exist_ok=True)
tifffile.imwrite(output_path, (full_res * 65535).astype(np.uint16))
print(f"Saved: {output_path}")

## Save & Load Configuration

In [ ]:
# Save configuration
CONFIG_NAME = "my_config"

config_path = save_config(
    weights=explorer.get_weights(),
    output_path=CONFIG_DIR / f"{CONFIG_NAME}.yaml",
    name=CONFIG_NAME,
    description="Membrane pseudochannel weights",
)
print(f"Saved: {config_path}")

In [ ]:
# List saved configs
for cfg in list_configs(CONFIG_DIR):
    print(f"  {cfg['name']}: {cfg['num_channels']} channels")

In [ ]:
# Load and apply a saved config
config = load_config(CONFIG_DIR / f"{CONFIG_NAME}.yaml")
explorer.set_weights(get_weights_from_config(config))
print(f"Loaded: {CONFIG_NAME}")

## Batch Processing

Apply saved weights to multiple images.

In [ ]:
# Batch process folder-based data
# output_paths = batch_process_directory(
#     root_path="../data",
#     config_path=CONFIG_DIR / f"{CONFIG_NAME}.yaml",
#     output_folder=OUTPUT_DIR / "batch",
# )
# print(f"Processed {len(output_paths)} folders")

In [ ]:
# Batch process OME-TIFF files
# output_paths = process_ome_tiff_batch(
#     tiff_files=["../data/sample1.ome.tiff", "../data/sample2.ome.tiff"],
#     marker_files="../data/markers.csv",
#     config_path=CONFIG_DIR / f"{CONFIG_NAME}.yaml",
#     output_folder=OUTPUT_DIR / "batch",
# )
# print(f"Processed {len(output_paths)} files")

In [ ]:
# Batch process MCMICRO experiments
# Recursively finds: .../background/image.ome.tiff + markers.csv
# Outputs to sibling folder: .../pseudochannel/pseudochannel.tif

# MCMICRO_ROOT = Path("/mnt/CEPH/Disco_analysis_station/staged_data/CRC")
# 
# # Preview what will be processed
# experiments = find_mcmicro_experiments(MCMICRO_ROOT)
# print(f"Found {len(experiments)} experiments:")
# for exp in experiments:
#     print(f"  {exp['experiment_path'].name}: {exp['image_path'].name}")
# 
# # Process all experiments
# output_paths = process_mcmicro_batch(
#     root_path=MCMICRO_ROOT,
#     config_path=CONFIG_DIR / f"{CONFIG_NAME}.yaml",
#     mcmicro_markers=True,  # Uses marker_name column, filters remove=TRUE
# )
# print(f"Processed {len(output_paths)} experiments")

## Batch Segmentation

After generating pseudochannels, segment them with Cellpose.

**Config options:**
- `config = explorer.get_cellpose_config()` — Use parameters tuned in the widget
- `config = "path/to/config.yaml"` — Extract cellpose section from YAML
- `config = CellposeConfig(diameter=30, ...)` — Specify directly
- `config = None` — Use defaults (auto GPU, cyto3 model)

In [ ]:
# Batch segment MCMICRO experiments
# Requires pseudochannel images to exist (run process_mcmicro_batch first)

# from pseudochannel import segment_mcmicro_batch, process_and_segment_mcmicro_batch
#
# # Option A: Use config from explorer (after tuning parameters)
# # cellpose_config = explorer.get_cellpose_config()
#
# # Option B: Use YAML config (if it has cellpose section)
# # Option C: Use defaults (just don't pass config)
#
# seg_outputs = segment_mcmicro_batch(
#     root_path=MCMICRO_ROOT,
#     config=CONFIG_DIR / f"{CONFIG_NAME}.yaml",  # or cellpose_config, or None
#     mcmicro_markers=True,
# )
# print(f"Segmented {len(seg_outputs)} experiments")
#
# # Or do both pseudochannel + segmentation in one call:
# # pseudo_paths, seg_paths = process_and_segment_mcmicro_batch(
# #     root_path=MCMICRO_ROOT,
# #     config_path=CONFIG_DIR / f"{CONFIG_NAME}.yaml",
# #     mcmicro_markers=True,
# # )